# CNN with transfer learning from InceptionV3 trained on Image net
### This is the implentation of step 6 and 7 in the dog_app notebook.

This is here so I can train and get the best CNN for classifying dog breeds to put into an app down the road...

### Includes

In [1]:
from keras.utils import np_utils
from sklearn.datasets import load_files  
import numpy as np
from glob import glob

import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline

Using TensorFlow backend.


### Human and dog detector functions

Human detector uses opencv, dog detector uses pretrained ResNet50 network.

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image as image_processor                 
from tqdm import tqdm


# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image_processor.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image_processor.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

def ResNet50_predict_labels(img_path):
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

def dog_detector(img_path):
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151)) 

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    return len(faces) > 0

### Get bottleneck features from the train dog pictures ran on the inception V3 network.

In [4]:
bottleneck_features = np.load('bottleneck_features/DogInceptionV3Data.npz')
train_InceptionV3 = bottleneck_features['train']
valid_InceptionV3 = bottleneck_features['valid']
test_InceptionV3 = bottleneck_features['test']

### Import dog labels from dataset, don't need features since we are using the features listed above:

In [5]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return dog_files, dog_targets

_, train_targets = load_dataset('dogImages/train')
_, valid_targets = load_dataset('dogImages/valid')
_, test_targets = load_dataset('dogImages/test')

dog_names = [item[20:-1] for item in sorted(glob("dogImages/train/*/"))]

### Model:

In [6]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

inception_model = Sequential()

inception_model.add(GlobalAveragePooling2D(input_shape=train_InceptionV3.shape[1:]))

#overfits heavily, two layers are good before output, better accuracy by increasing number of nodes for first fc
inception_model.add(Dense(3072, activation='relu'))
inception_model.add(Dropout(0.5))
inception_model.add(Dense(512, activation='relu'))#leave this 512, any bigger causes too much overfitting
inception_model.add(Dropout(0.3))
inception_model.add(Dense(133, activation='softmax'))

inception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3072)              6294528   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3072)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 133)               68229     
Total params: 7,936,133
Trainable params: 7,936,133
Non-trainable params: 0
_________________________________________________________________


### Model compiliation:

In [7]:
from keras import optimizers as opt 
#need low learning rate to help overfitting, takes forever to train tho
inception_model.compile(loss='categorical_crossentropy', optimizer=opt.RMSprop(lr=0.000001), 
                        metrics=['accuracy'])

### Training:

In [95]:
from keras.callbacks import ModelCheckpoint  
#from keras.preprocessing.image import ImageDataGenerator #for data augmentation

epochs = 10
batch_size = 5


checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.transfer.inception.test.hdf5', 
                               verbose=1, save_best_only=True)
                           
inception_model.fit(train_InceptionV3, train_targets, validation_data=(valid_InceptionV3, valid_targets),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6680 samples, validate on 835 samples
Epoch 1/10
6680/6680 [==============================] - 12s 2ms/step - loss: 0.4096 - acc: 0.8744 - val_loss: 0.4751 - val_acc: 0.8599

Epoch 00001: val_loss improved from inf to 0.47511, saving model to saved_models/weights.best.transfer.inception.hdf5
Epoch 2/10
6680/6680 [==============================] - 12s 2ms/step - loss: 0.4000 - acc: 0.8753 - val_loss: 0.4762 - val_acc: 0.8575

Epoch 00002: val_loss did not improve from 0.47511
Epoch 3/10
6680/6680 [==============================] - 12s 2ms/step - loss: 0.4081 - acc: 0.8744 - val_loss: 0.4760 - val_acc: 0.8563

Epoch 00003: val_loss did not improve from 0.47511
Epoch 4/10
6680/6680 [==============================] - 12s 2ms/step - loss: 0.4150 - acc: 0.8744 - val_loss: 0.4758 - val_acc: 0.8563

Epoch 00004: val_loss did not improve from 0.47511
Epoch 5/10
6680/6680 [==============================] - 12s 2ms/step - loss: 0.4055 - acc: 0.8757 - val_loss: 0.4764 - val_acc: 0.8575

Ep

### Load best validation loss weights:
Pretty much the best I am going to get without some super huge networks and large training times.

In [8]:
inception_model.load_weights('saved_models/weights.best.transfer.inception.hdf5')

### Test model accuracy:

In [97]:
inception_predictions = [np.argmax(inception_model.predict(
                        np.expand_dims(feature, axis=0))) for feature in test_InceptionV3]

test_accuracy = 100*np.sum(np.array(inception_predictions)==np.argmax(test_targets, axis=1))/len(inception_predictions)
print('Test accuracy: {0:.2f}'.format(test_accuracy))

Test accuracy: 83.13


### Predicting dog breeds using trained model functions:

In [9]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input as pre_process_input
inception_transferred = InceptionV3(weights='imagenet', include_top=False)

#extract_bottleneck_features.py recreated the net everytime function was called, causing huge slowdowns and higher
#memory usage with each call, should only be ran once to create the object
def extract_InceptionV3_2(tensor):
    return inception_transferred.predict(pre_process_input(tensor))

def inception_predict_breed(image_path):
    botneck_feature = extract_InceptionV3_2(path_to_tensor(image_path))
    predicted_breed = np.argmax(inception_model.predict(botneck_feature))
    
    return dog_names[predicted_breed]

### Classifier using trained model and dog/human detectors

In [10]:
import matplotlib.image as mpimg

def classifer(image_path):
    display_img = np.array(mpimg.imread(image_path), dtype=float)
    if face_detector(image_path) and not dog_detector(image_path):
        print("Hello human!")
        plt.imshow(display_img)
        plt.show()
        print("You look like a {} dog!!".format(inception_predict_breed(image_path)))
    elif dog_detector(image_path) and not face_detector(image_path):
        print("Hello dog!")
        plt.imshow(display_img)
        plt.show()
        print("You are a {} dog!! {}/10".format(inception_predict_breed(image_path), np.random.randint(
                                                                                     low=10, high=15, size=None)))
    else:
        print("Hello thing!")
        plt.imshow(display_img)
        plt.show()
        print("I don't know what you are, please try another picture!")
        print("If I wrong, then you may be a {}".format(inception_predict_breed(image_path)))

### Grab all images in images/step7_test_images and run the classifier on them:

In [ ]:
sample_images = np.array(glob('images/step7_test_images/*'))
np.random.shuffle(sample_images)

for sample_image in sample_images:
    classifer(sample_image)
    print()